In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 20 09:18:43 2020

@author: Abdul Qayyum
"""


import os
import torch
from torch.utils.data import Dataset
import cv2
#from Code.utils.onehot import onehot

import numpy as np
from torchvision import transforms

def onehot(data, n):
    """onehot ecoder"""
    buf = np.zeros(data.shape + (n,))
    nmsk = np.arange(data.size) * n + data.ravel()
    buf.ravel()[nmsk - 1] = 1
    return buf
def convert_to_one_hot(volume, class_number):
    '''
    one hot??
    :param volume: label
    :param C: class number
    :return:
    '''
    shape = [class_number]+list(volume.shape)
    volume_one = np.eye(class_number)[volume.reshape(-1)].T
    volume_one = volume_one.reshape(shape)
    return volume_one

#########################################################
########################################################## dataset loader ######################################
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
#import keras
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import os
import torch
from torch.utils.data import Dataset
import cv2

class Dataset(BaseDataset):
    """Emidec Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['myocardium', 'myocardiuminfec','bloodpool','noreflow','unlabelled']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            transform=None
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        self.transform=transform
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_name = self.ids[i]
        #mask = cv2.imread(self.masks_fps[i], 0)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
            mask=mask.transpose(2,0,1) # n_class*w*H
            mask=torch.FloatTensor(mask)
            #mask=mask.transpose(2,0,1) # n_class*w*H
        #onehot_label=torch.FloatTensor(img_label_onehot)
        #print(onehot_label.shape)
            
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        if self.transform:
            image=self.transform(image)
            
        return image, mask,image_name
        
    def __len__(self):
        return len(self.ids)
    
imagepath='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/StructSegdataset/emidecdataset/Trainimagenew/'
maskpath='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/StructSegdataset/emidecdataset/Trainmasksequence/'    
x_train_dir=imagepath
y_train_dir=maskpath
#data=data[:,:,0]  
# Lets look at data we have
from torchvision import transforms
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485,0.456,0.406],
                                                  std=[0.229,0.224,0.225])])
#dataset = Dataset(x_train_dir, y_train_dir, classes=['leftlung', 'rightlung', 'disease'],transform=transform)
#dataset = Dataset(x_train_dir, y_train_dir, classes=['leftlung', 'rightlung', 'disease'],transform=transform)

#image, mask = dataset[4] # get some sample
#print(image.shape)
#print(mask.shape)
#batch_size=1
from torch.utils.data import DataLoader
#train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
#for train_batch in train_dataloader:
#    print(train_batch[0].shape)
#    print(train_batch[1].shape)
#    break
#transform=transforms.Compose([transforms.ToTensor(),
#                              transforms.Normalize(mean=[0.485,0.456,0.406],
#                                                  std=[0.229,0.224,0.225])])

#imagepath='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/Imgs/'
#maskpath='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/GT/'


#train_dataset=Mydataset(imagepath,maskpath,transform)

# batch_size=4
# from torch.utils.data import DataLoader
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# for train_batch in train_dataloader:
#     print(train_batch)

In [ ]:
#################################################################segmentation model resnet ################################
import torch
import torch.nn as nn
import torch.nn.functional as F


class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1)
            
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class one_conv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(one_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, 1)    
        )
    def forward(self,x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.pool = nn.MaxPool2d(2)
        self.mpconv = double_conv(in_ch, out_ch)
        self.bridge = one_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.pool(x)
        #print(x.shape)
        x_1 = self.mpconv(x)
        #print(x_1.shape)
        x_2 = self.bridge(x)
        #print(x_2.shape)
        x = x_1 + x_2
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)
        self.bridge = one_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x) + self.bridge(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, 1)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class ResUNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(ResUNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        # x = F.softmax(x, dim=1)
        return x
    
modelUres=ResUNet(3,5)
print(modelUres)

In [ ]:
##################################################training path######################################
import os
import numpy as np
import torch.optim as optim
#from Code.utils.dataloader_MulClsLungInf_UNet import LungDataset
from torchvision import transforms
# from LungData import test_dataloader, train_dataloader  # pls change batch_size
from torch.utils.data import DataLoader
#from Code.model_lung_infection.InfNet_UNet import *


def train(epo_num, n_classes, n_channels, batch_size, lr, save_path):
    #dataset = Dataset(x_train_dir, y_train_dir, classes=['leftlung', 'rightlung', 'disease'],transform=transform)
    dataset = Dataset(x_train_dir, y_train_dir, classes=['myocardium', 'myocardiuminfec','bloodpool','noreflow'],transform=transform)
    
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modelUres=ResUNet(n_channels,n_classes)
    modelUres=modelUres.to(device)

    criterion = nn.BCELoss().to(device)
    #optimizer = optim.SGD(lung_model.parameters(), lr=lr, momentum=0.7)
    optimizer = optim.SGD(modelUres.parameters(), lr=lr, momentum=0.7)

    print("#" * 20, "\nStart Training (network-Net)\ "
                    "\n", "#" * 20)

    for epo in range(epo_num):

        train_loss = 0
        #lung_model.train()
        modelUres.train()

        for index, (img, img_mask, _) in enumerate(train_dataloader):

            img = img.to(device)
            #pseudo = pseudo.to(device)
            img_mask = img_mask.to(device)
            #img_mask=np.transpose(img_mask, (0, 2, 1, 3))

            optimizer.zero_grad()
            #utput = lung_model(img)
            output=modelUres(img)

            output = torch.sigmoid(output)  # output.shape is torch.Size([4, 2, 160, 160])
            loss = criterion(output, img_mask)

            loss.backward()
            iter_loss = loss.item()
            train_loss += iter_loss
            optimizer.step()

            if np.mod(index, 20) == 0:
                print('Epoch: {}/{}, Step: {}/{}, Train loss is {}'.format(epo, epo_num, index, len(train_dataloader), iter_loss))

        os.makedirs('./checkpoints//UNet_Multi-Class', exist_ok=True)
        if np.mod(epo+1, 10) == 0:
            torch.save(modelUres.state_dict(),
                       './Snapshots/save_weightsheart/{}/unet_model_{}.pkl'.format(save_path, epo+1))
            print('Saving checkpoints: unet_model_{}.pkl'.format(epo+1))

train(epo_num=300,n_classes=5,n_channels=3,batch_size=16,lr=1e-2,save_path='save_trained_weights')

In [ ]:
snapshot_dir='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/Snapshots/save_weightsheart/save_trained_weights/unet_model_100.pkl'
from tqdm import tqdm
num_classes=3
import natsort
import matplotlib.pyplot as plt
import natsort
import matplotlib.pyplot as plt
from skimage import io, exposure, img_as_uint, img_as_float
import imageio
import numpy as np
from PIL import Image
import cv2
import numpy as np
from PIL import Image
import cv2
from skimage.transform import resize
#from keras.models import Model
#import nibabel as nib
import cv2
import numpy as np
import os
import natsort
                
##############################################################load trained model ############################################

input_channels=3
num_classes=5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#lung_model = Inf_Net_UNet(input_channels, num_classes).cuda()
modelUres =ResUNet(3,5).cuda()
modelUres.load_state_dict(torch.load(snapshot_dir))
modelUres.eval()

In [ ]:
################################################################save image as case folder orpatients #############################################
path1='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/StructSegdataset/emidecdataset/testimages/'
oslist=os.listdir(path1)
filesnew=natsort.natsorted(oslist)
path_savemask='/raid/Home/Users/aqayyum/pymultimodel/Covid19Segmentationmodels/ModelsPredictions/predictemidec/'
for i, volume in enumerate(filesnew):
    print(volume)
    cur_path = os.path.join(path1, volume)
    files=natsort.natsorted(os.listdir(cur_path))
    alist=[]
    for n, id_ in tqdm(enumerate(files), total=len(files)):
        print(n)
        print(id_)
        #img=imageio.imread(os.path.join(cur_path,id_))
        img=Image.open(os.path.join(cur_path,id_))
        #print(img)
        img= transform(img)
        img=img.to(device)
        #print(type(img1))
        img1=img.unsqueeze(0)
        #output=lung_model(img1)
        output = modelUres(img1)
        #output1 = lung_model(img1)
        output = torch.sigmoid(output)  # output.shape is torch.Size([4, 2, 160, 160])
        b, _, w, h = output.size()
        print(output.size())
        pred = output.cpu().permute(0, 2, 3, 1).contiguous().view(-1, num_classes).max(1)[1].view(b, w, h).numpy().squeeze()
        #pred = output.cpu().contiguous().view(-1, num_classes).max(1)[1].view(b, w, h).numpy().squeeze()
        #print('Class numbers of prediction in total:', np.unique(pred))
        # pred = misc.imresize(pred, size=(w_gt, h_gt))
        #os.makedirs(save_path, exist_ok=True)
        savpath=os.path.join(path_savemask,str(volume))
        savefold=createFolder(savpath)
        #io.imsave(savpath+'/'+str(n)+'.png',pred)
        cv2.imwrite(savpath+'/'+str(n)+'.png',pred)
        alist.append(pred)  


########################################################
print('done or complete')